Getting Started

Let’s get right down to business and define our first form:

In [1]:
from wtforms import Form, BooleanField, StringField, validators,DateTimeField,TextAreaField,IntegerField,DecimalField, PasswordField

class RegistrationForm(Form):
    username     = StringField('Username', [validators.Length(min=4, max=25)])
    email        = StringField('Email Address', [validators.Length(min=6, max=35)])
    accept_rules = BooleanField('I accept the site rules', [validators.InputRequired()])

When you create a form, you define the fields in a way that is similar to the way many ORM’s have you define their columns: By defining class variables which are instantiations of the fields.  Because forms are regular Python classes, you can easily extend them as you would expect:


In [2]:
class ProfileForm(Form):
    birthday  = DateTimeField('Your Birthday', format='%m/%d/%y')
    signature = TextAreaField('Forum Signature')

class AdminProfileForm(ProfileForm):
    username = StringField('Username', [validators.Length(max=40)])
    level    = IntegerField('User Level', [validators.NumberRange(min=0, max=10)])

Via subclassing, AdminProfileForm gains all the fields already defined in ProfileForm. This allows you to easily share common subsets of fields between forms, such as the example above, where we are adding admin-only fields to ProfileForm.
Using Forms

Using a form is as simple as instantiating it. Consider the following django-like view, using the RegistrationForm we defined earlier:

In [3]:
def register(request):
    form = RegistrationForm(request.POST)
    if request.method == 'POST' and form.validate():
        user = User()
        user.username = form.username.data
        user.email = form.email.data
        user.save()
        redirect('register')
    return render_response('register.html', form=form)

First, we instantiate the form, providing it with any data available in request.POST. We then check if the request is made using POST, and if it is, we validate the form, and check that the user accepted the rules. If successful, we create a new User and assign the data from the validated form to it, and save it.
Editing existing objects

Our earlier registration example showed how to accept input and validate it for new entries, but what if we want to edit an existing object? Easy:

In [4]:
def edit_profile(request):
    user = request.current_user
    form = ProfileForm(request.POST, user)
    if request.method == 'POST' and form.validate():
        form.populate_obj(user)
        user.save()
        redirect('edit_profile')
    return render_response('edit_profile.html', form=form)

Here, we instantiate the form by providing both request.POST and the user object to the form. By doing this, the form will get any data that isn’t present in the post data from the user object.

We’re also using the form’s populate_obj method to re-populate the user object with the contents of the validated form. This method is provided for convenience, for use when the field names match the names on the object you’re providing with data. Typically, you will want to assign the values manually, but for this simple case it’s perfect. It can also be useful for CRUD and admin forms.
Exploring in the console

WTForms forms are very simple container objects, and perhaps the easiest way to find out what’s available to you in a form is to play around with a form in the console:

In [5]:
from wtforms import Form, StringField, validators

class UsernameForm(Form):

    username = StringField('Username', [validators.Length(min=5)], default='test')

In [6]:
form = UsernameForm()

form['username']
<wtforms.fields.core.StringField object at ...>

form.username.data
'test'

form.validate()
False

form.errors

In [7]:
{'username': ['Field must be at least 5 characters long.']}

{'username': ['Field must be at least 5 characters long.']}

What we’ve found here is that when you instantiate a form, it contains instances of all the fields, which can be accessed via either dictionary-style or attribute-style. These fields have their own properties, as does the enclosing form.

When we validate the form, it returns False, meaning at least one validator was not satisfied. form.errors will give you a summary of all the errors.

In [8]:
form2 = UsernameForm(username='Robert')

form2.data
form2.validate()

True

{'username': 'Robert'}


True

This time, we passed a new value for username when instantiating UserForm, and it was sufficient to validate the form.
How Forms get data

In addition to providing data using the first two arguments (formdata and obj), you can pass keyword arguments to populate the form. Note though that a few names are reserved: formdata, obj, and prefix.

formdata takes precendence over obj, which itself takes precedence over keyword arguments. For example:

In [9]:
def change_username(request):
    user = request.current_user
    form = ChangeUsernameForm(request.POST, user, username='silly')
    if request.method == 'POST' and form.validate():
        user.username = form.username.data
        user.save()
        return redirect('change_username')
    return render_response('change_username.html', form=form)

While you almost never use all three methods together in practice, it illustrates how WTForms looks up the username field:

    If a form was submitted (request.POST is not empty), process the form input. Even if there was no form input for this field in particular, if there exists form input of any sort, then we will process the form input.

    If there was no form input, then try the following in order:

        Check if user has an attribute named username.

        Check if a keyword argument named username was provided.

        Finally, if everything else fails, use the default value provided by the field, if any.

Validators

Validation in WTForms is done by providing a field with a set of validators to run when the containing form is validated. You provide these via the field constructor’s second argument, validators:

In [10]:
class ChangeEmailForm(Form):
    email = StringField('Email', [validators.Length(min=6, max=120), validators.Email()])

You can provide any number of validators to a field. Typically, you will want to provide a custom error message:

In [11]:
class ChangeEmailForm(Form):
    email = StringField('Email', [
        validators.Length(min=6, message=_('Little short for an email address?')),
        validators.Email(message=_('That\'s not a valid email address.'))
    ])

TypeError: 'bool' object is not callable

It is generally preferable to provide your own messages, as the default messages by necessity are generic. This is also the way to provide localised error messages.

For a list of all the built-in validators, check the Validators Reference
Rendering Fields

Rendering a field is as simple as coercing it to a string:

In [ ]:
from wtforms import Form, StringField

class SimpleForm(Form):

  content = StringField('content')


form = SimpleForm(content='foobar')

str(form.content)

Markup('<input id="content" name="content" type="text" value="foobar">')

However, the real power comes from rendering the field with its __call__() method. By calling the field, you can provide keyword arguments, which will be injected as html attributes in the output:

In [ ]:
form.content(style="width: 200px;", class_="bar")

In [ ]:
Markup('<input class="bar" id="content" name="content" style="width: 200px;" type="text" value="foobar">')

Now let’s apply this power to rendering a form in a Jinja template. First, our form:

In [ ]:
class LoginForm(Form):
    username = StringField('Username')
    password = PasswordField('Password')

form = LoginForm()

And the template:

<form method="POST" action="/login">
    <div>{{ form.username.label }}: {{ form.username(class="css_class") }}</div>
    <div>{{ form.password.label }}: {{ form.password() }}</div>
</form>

Alternately, if you’re using Django templates, you can use the form_field templatetag we provide in our Django extension, when you want to pass keyword arguments:

{% load wtforms %}
<form method="POST" action="/login">
    <div>
        {{ form.username.label }}:
        {% form_field form.username class="css_class" %}
    </div>
    <div>
        {{ form.password.label }}:
        {{ form.password }}
    </div>
</form>

Both of these will output:

<form method="POST" action="/login">
    <div>
        <label for="username">Username</label>:
        <input class="css_class" id="username" name="username" type="text" value="" />
    </div>
    <div>
        <label for="password">Password</label>:
        <input id="password" name="password" type="password" value="" />
    </div>
</form>

WTForms is template engine agnostic, and will work with anything that allows attribute access, string coercion, and/or function calls. The form_field templatetag is provided as a convenience as you can’t pass arguments in Django templates.
Displaying Errors

Now that we have a template for our form, let’s add error messages:

<form method="POST" action="/login">
    <div>{{ form.username.label }}: {{ form.username(class="css_class") }}</div>
    {% if form.username.errors %}
        <ul class="errors">{% for error in form.username.errors %}<li>{{ error }}</li>{% endfor %}</ul>
    {% endif %}

    <div>{{ form.password.label }}: {{ form.password() }}</div>
    {% if form.password.errors %}
        <ul class="errors">{% for error in form.password.errors %}<li>{{ error }}</li>{% endfor %}</ul>
    {% endif %}
</form>

If you prefer one big list of errors at the top, this is also easy:

{% if form.errors %}
    <ul class="errors">
        {% for field_name, field_errors in form.errors|dictsort if field_errors %}
            {% for error in field_errors %}
                <li>{{ form[field_name].label }}: {{ error }}</li>
            {% endfor %}
        {% endfor %}
    </ul>
{% endif %}

As error handling can become a rather verbose affair, it is preferable to use Jinja macros (or equivalent) to reduce boilerplate in your templates. (example)
Custom Validators

There are two ways to provide custom validators. By defining a custom validator and using it on a field:

In [ ]:
from wtforms.validators import ValidationError

def is_42(form, field):
    if field.data != 42:
        raise ValidationError('Must be 42')

class FourtyTwoForm(Form):
    num = IntegerField('Number', [is_42])

Or by providing an in-form field-specific validator:

In [ ]:
class FourtyTwoForm(Form):
    num = IntegerField('Number')

    def validate_num(form, field):
        if field.data != 42:
            raise ValidationError('Must be 42')

For more complex validators that take parameters, check the Custom validators section.